In [ ]:
# Homework 5
# Testing the avalanche properties of AES using an automated code

# Write a function to test the avalanche property of AES when different single bits of 
# the inputblock are inverted. 

# In this experiment you receive an initial inputblock and key to perform AES encryption and
# find the cipherblock. Then you will use the bitlist provided as the 3rd input to the function
# to decide which bit of the inputblock to invert in additional experiments and then perform the
# AES encryption again on the modified input (with only one bit difference to original input)
# Then you compare the ciphertext for each additional experiment with the ciphertext of the
# original experiment and count the number of bits that are different between them.

# The output of your function will be the list of the number-of-differences from all experiments

# Make sure you test your code before submission by setting the bitlist to [7] and comparing the results 
# with Table 6.5 and Table 6.6 of the textbook. you can also try inverting other bit values manually and generate 
# ciphers using the AES example we did in the class and compare with the output of your function.

# As usual, we will import your submitted python file in another script and just call the aes_input_av_test() and
# aes_key_av_test() functions and check the output list of each function. It's ok if you have other utility functions
# in your submission and they will not be called or tested.
# plainblock = int(temp, 2).to_bytes(len(temp) // 8, byteorder='big')

In [2]:
def byteseq2binstr(byteseq):
    # first convert to a list string binary representations of each byte
    bitslist2 = [bin(int(b))[2:].zfill(8) for b in byteseq]
    
    # then merge all those strings
    allbitsstr = ''.join(bitslist2)
    
    return allbitsstr

In [25]:
def XORbits(bitstr1,bitstr2):
    # Both bit strings should be the same length
    # output will be a string with the same length
    B1 = [int(q) for q in bitstr1]
    B2 = [int(w) for w in bitstr2]
    xor_result = ''.join([str(d^f) for d,f in zip(B1, B2)])
    
    return xor_result

In [26]:
b = 7
a = '0'*(b) + '1' + '0'*(7-b)
print(a)

00000001


In [31]:
def invertbit(inputblock, b):
    inputblockstr = byteseq2binstr(inputblock)
    xorstr = '0'*(b) + '1' + '0'*(127-b)
    res = XORbits(inputblockstr, xorstr)
    return int(res, 2).to_bytes(len(res) // 8, byteorder='big')

invertbit(b'\x00\x01\x01\x01\x01\x01\x01\x01\x00\x01\x01\x01\x01\x01\x01\x01', 127)

b'\x00\x01\x01\x01\x01\x01\x01\x01\x00\x01\x01\x01\x01\x01\x01\x00'

In [39]:
def findbitdiff(originalcipher, newcipher):
    orig = byteseq2binstr(originalcipher)
    new = byteseq2binstr(newcipher)
    count = 0
    for i in range(0, len(orig)):
        if orig[i] != new[i]:
            count = count+1
    
    return count

findbitdiff(b'\xff\xff', b'\x00\x00')

16

In [3]:
from Crypto.Cipher import AES
key = b'Sixteen byte key'
msg = b'1234567890123456'
cipher = AES.new(key, AES.MODE_ECB)
enc_msg = cipher.encrypt(msg)
print(byteseq2binstr(enc_msg))
dec_msg = cipher.decrypt(enc_msg)
print(byteseq2binstr(dec_msg))

10000111101001101100100010011010101100010000011001101100111101101001010101110110000010100111100001000111111111100101001000110100
00110001001100100011001100110100001101010011011000110111001110000011100100110000001100010011001000110011001101000011010100110110


In [40]:
def aes_input_av_test(inputblock, key, bitlist):
    # inputblock and key are 16 byte values each
    # bitlist is a list of integers that define the position of the
    # bit in the inputblock that needs to be inverted, one at a time, for example
    # [0, 3, 6, 25, 78, 127]
    
    # 1- any initializations necessary
    diff_list = []
    
    # 2- perform encryption of the original values
    #    anyway you like. It doesn't have to be with 
    #    with this exact function form
    cipher = AES.new(key, AES.MODE_ECB)
    originalcipher = cipher.encrypt(inputblock)
    
    # 3- for every value given in the bitlist:
    for b in bitlist:
        #invert the value of the corresponding bit in the inputblock (doesn't have to be in this exact
        # function form)
        newinput = invertbit(inputblock, b)
        
        # perform encryption
        newcipher = cipher.encrypt(newinput)
        
        # find the number of bit differences between the two ciphertexts (doesn't have to be exactly in
        # this function form)
        # Use any method you like to find the difference. 
        numbitdifferences = findbitdiff (originalcipher, newcipher)
        
        # add it to the list
        diff_list.append(numbitdifferences)
        
    # return the list of numbers
    return diff_list

aes_input_av_test(b'\x01\x23\x45\x67\x89\xab\xcd\xef\xfe\xdc\xba\x98\x76\x54\x32\x10', b'\x0f\x15\x71\xc9\x47\xd9\xe8\x59\x0c\xb7\xad\xd6\xaf\x7f\x67\x98', [0, 7, 6, 25, 78, 127])

[62, 58, 68, 66, 65, 65]

In [41]:
# We also perform similar experiment by keeping the inputblocl fixed and changing the
# selected bits of the key
def aes_key_av_test(inputblock, key, bitlist):
    # inputblock and key are 16 byte values each
    # bitlist is a list of integers that define the position of the
    # bit in the key that needs to be inverted, one at a time, for example
    # [0, 3, 6, 25, 78, 127]
    
    # 1- any initializations necessary
    diff_list = []
    
    # 2- perform encryption of the original values
    #    anyway you like. It doesn't have to be with 
    #    with this exact function form
    cipher = AES.new(key, AES.MODE_ECB)
    originalcipher = cipher.encrypt(inputblock)
    
    # 3- for every value given in the bitlist:
    for b in bitlist:
        #invert the value of the corresponding bit in the key (doesn't have to be in this exact
        # function form)
        newkey = invertbit(key, b)
        cipher = AES.new(newkey, AES.MODE_ECB)
        # perform encryption
        newcipher = cipher.encrypt(inputblock)
        
        # find the number of bit differences between the two ciphertexts (doesn't have to be exactly in
        # this function form)
        numbitdifferences = findbitdiff (originalcipher, newcipher)
        
        # add it to the list
        diff_list.append(numbitdifferences)
        
    # return the list of numbers
    return diff_list

# aes_key_av_test(b'\x01\x23\x45\x67\x89\xab\xcd\xef\xfe\xdc\xba\x98\x76\x54\x32\x10', b'\x0f\x15\x71\xc9\x47\xd9\xe8\x59\x0c\xb7\xad\xd6\xaf\x7f\x67\x98', [0, 7, 6, 25, 78, 127])

    
    
    

[61, 53, 62, 70, 70, 67]